# Exercício no 1
## “Monitor com Threads”

O objetivo deste exercício é criar um código multithread em Python capaz de monitorar
alterações em sites de notícias. Suponha os seguintes 4 sites brasileiros de notícias:

- https://g1.globo.com
- https://noticias.uol.com.br
- https://www.r7.com
- https://www.cnnbrasil.com.br
 
O código deverá inicializar 4 threads, um para cada site.
A função de monitoramento deverá receber a **URL**, **INTERVALO DE VERIFICAÇÕES** e a
**QUANTIDADE DE VERIFICAÇÕES**. Ou seja, estes três parâmetros devem ser enviados para a
função associada a cada thread.

> Ao término do processamento, cada thread deverá dizer se o site foi modificado ou não no
período monitorado.

In [1]:
import requests
import hashlib
import pandas as pd

from time import sleep, perf_counter
from datetime import datetime

# SiteMonitor 
import sys; sys.path.append('..')
import site_monitoring as sm

In [2]:
# # Constants
# # MARK: - Parameters
# SITES = ['https://g1.globo.com/',
#          'https://noticias.uol.com.br/',
#          'https://www.r7.com/',
#          'https://www.cnnbrasil.com.br/']

# MONITORING_INTERVAL = 2
# MONITORING_COUNT_TARGET = 5

# # MARK: - Utils
# HEADERS = {'User-Agent': 'Mozilla/5.0'}

# class HashComparer:
#     def __init__(self):
#         self.previous = None
#         self.latest = None

#     def update(self, new_hash):
#         self.previous = self.latest
#         self.latest = new_hash   

#     def did_change(self):
#         if self.previous is not None and self.latest is not None:
#             return self.previous != self.latest
#         return False

# class SiteMonitor:
#     def __init__(self):
#         self._data = pd.DataFrame(columns = ['timestamp', 'attempt', 'url', 'changed', 'latest', 'previous'])
    
#     def monitor(self, site: str):
        
#         hash_comparer = HashComparer()

#         for count in range(MONITORING_COUNT_TARGET): # Count from 0 to TARGET

#             response = requests.get(
#                 site,
#                 headers = HEADERS
#             )

#             utf8_content = repr(response.text).encode('utf-8')
#             response_hash = hashlib.sha224(utf8_content).hexdigest()

#             hash_comparer.update(response_hash)

#             self._data = pd.concat([self._data, pd.DataFrame({
#                 'timestamp': [datetime.now()],
#                 'attempt': [count],
#                 'url': [site],
#                 'changed': [hash_comparer.did_change()],
#                 'latest': [hash_comparer.latest],
#                 'previous': [hash_comparer.previous]
#             })], ignore_index=True)

#             print(f'[{datetime.now()}] Monitoring attempt: ({count}) Url: {site} Did change?: {hash_comparer.did_change()} | Hashes latest: {hash_comparer.latest} VS. previous: {hash_comparer.previous}')

#             sleep(MONITORING_INTERVAL)


## 1 Sequential

In [3]:
sites_data = []

start_time = perf_counter()

for site in sm.SITES: # For each URL
    site_monitor = sm.SiteMonitor()
    data = site_monitor.monitor_and_get_data(site)

    sites_data.append(data)

end_time = perf_counter()

sequential_elapsed_time = end_time - start_time

print(f'As tarefas levaram {sequential_elapsed_time: 0.2f} segundo(s) para executar.')

/workspaces/parallel-and-distribute-computing/unit2/exercises/1-site-monitoring/../site_monitoring.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._data = pd.concat([self._data, pd.DataFrame({


[2025-09-09 23:12:02.501989] Monitoring attempt: (0) Url: https://g1.globo.com/ Did change?: False | Hashes latest: 0c38b074a40bf19e69e1529def5b42636ce914be929bf34293c72d94 VS. previous: None
[2025-09-09 23:12:05.339644] Monitoring attempt: (1) Url: https://g1.globo.com/ Did change?: True | Hashes latest: db596588332cb0e98d18fdb8770cc6b806dc398f5bc6b052a90d64d3 VS. previous: 0c38b074a40bf19e69e1529def5b42636ce914be929bf34293c72d94
[2025-09-09 23:12:08.167462] Monitoring attempt: (2) Url: https://g1.globo.com/ Did change?: True | Hashes latest: 06806f6b6b798b59dac9d9c201c7a15b98c04c1ed17cb90b1812bd53 VS. previous: db596588332cb0e98d18fdb8770cc6b806dc398f5bc6b052a90d64d3
[2025-09-09 23:12:11.018889] Monitoring attempt: (3) Url: https://g1.globo.com/ Did change?: True | Hashes latest: db596588332cb0e98d18fdb8770cc6b806dc398f5bc6b052a90d64d3 VS. previous: 06806f6b6b798b59dac9d9c201c7a15b98c04c1ed17cb90b1812bd53
[2025-09-09 23:12:13.849575] Monitoring attempt: (4) Url: https://g1.globo.com/

/workspaces/parallel-and-distribute-computing/unit2/exercises/1-site-monitoring/../site_monitoring.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._data = pd.concat([self._data, pd.DataFrame({


[2025-09-09 23:12:16.041010] Monitoring attempt: (0) Url: https://noticias.uol.com.br/ Did change?: False | Hashes latest: 6e2446858e57c491c1881553913c147cbae05b493bf32d61ab148089 VS. previous: None
[2025-09-09 23:12:18.098768] Monitoring attempt: (1) Url: https://noticias.uol.com.br/ Did change?: True | Hashes latest: 4c0380463fc4f8625fae567301070bf9493a26b41df49998b9020f78 VS. previous: 6e2446858e57c491c1881553913c147cbae05b493bf32d61ab148089
[2025-09-09 23:12:20.145247] Monitoring attempt: (2) Url: https://noticias.uol.com.br/ Did change?: True | Hashes latest: 7a681ed4fc9789db70acd971f1213a65e4e056d9e069a117409088f1 VS. previous: 4c0380463fc4f8625fae567301070bf9493a26b41df49998b9020f78
[2025-09-09 23:12:22.190413] Monitoring attempt: (3) Url: https://noticias.uol.com.br/ Did change?: True | Hashes latest: e6529d7c6c0532eaef88d900d0ce99303ebe1aba86bff52395b0a48d VS. previous: 7a681ed4fc9789db70acd971f1213a65e4e056d9e069a117409088f1
[2025-09-09 23:12:24.229621] Monitoring attempt: (4

/workspaces/parallel-and-distribute-computing/unit2/exercises/1-site-monitoring/../site_monitoring.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._data = pd.concat([self._data, pd.DataFrame({


[2025-09-09 23:12:26.525797] Monitoring attempt: (0) Url: https://www.r7.com/ Did change?: False | Hashes latest: 2370c944e035bcc3d352d7de743b8d66693b50253f46066e4e2a5f7a VS. previous: None
[2025-09-09 23:12:28.770960] Monitoring attempt: (1) Url: https://www.r7.com/ Did change?: True | Hashes latest: b62ba540fb88cad6b1db0eba2d9bc1b380eb51c6bb4b630017f48d67 VS. previous: 2370c944e035bcc3d352d7de743b8d66693b50253f46066e4e2a5f7a
[2025-09-09 23:12:31.221526] Monitoring attempt: (2) Url: https://www.r7.com/ Did change?: True | Hashes latest: 9fb1dc70451f90d0849a1767fb032a84267eae34146b718e53a3c101 VS. previous: b62ba540fb88cad6b1db0eba2d9bc1b380eb51c6bb4b630017f48d67
[2025-09-09 23:12:33.367473] Monitoring attempt: (3) Url: https://www.r7.com/ Did change?: True | Hashes latest: 396de9db562c4da486a0605364fc66da4ace890eef4fa14e85ba6a27 VS. previous: 9fb1dc70451f90d0849a1767fb032a84267eae34146b718e53a3c101
[2025-09-09 23:12:35.584295] Monitoring attempt: (4) Url: https://www.r7.com/ Did chang

/workspaces/parallel-and-distribute-computing/unit2/exercises/1-site-monitoring/../site_monitoring.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._data = pd.concat([self._data, pd.DataFrame({


[2025-09-09 23:12:37.917507] Monitoring attempt: (0) Url: https://www.cnnbrasil.com.br/ Did change?: False | Hashes latest: 4f0775ea7f765cb4040d4741d2c9f4c6c12e70dec5066fbd1be43106 VS. previous: None
[2025-09-09 23:12:40.668088] Monitoring attempt: (1) Url: https://www.cnnbrasil.com.br/ Did change?: True | Hashes latest: d10dbdcffe84505df38a4f7d2f6de2eaa1b6fba1930fc6c30305a59e VS. previous: 4f0775ea7f765cb4040d4741d2c9f4c6c12e70dec5066fbd1be43106
[2025-09-09 23:12:42.708263] Monitoring attempt: (2) Url: https://www.cnnbrasil.com.br/ Did change?: False | Hashes latest: d10dbdcffe84505df38a4f7d2f6de2eaa1b6fba1930fc6c30305a59e VS. previous: d10dbdcffe84505df38a4f7d2f6de2eaa1b6fba1930fc6c30305a59e
[2025-09-09 23:12:44.892403] Monitoring attempt: (3) Url: https://www.cnnbrasil.com.br/ Did change?: True | Hashes latest: f3f821bacb4f670b1bc461805cd848f6bae13b84be094c36380e40da VS. previous: d10dbdcffe84505df38a4f7d2f6de2eaa1b6fba1930fc6c30305a59e
[2025-09-09 23:12:46.933109] Monitoring attemp

In [4]:
for data in sites_data:
    display(data)

,timestamp,attempt,url,changed,latest,previous
0,2025-09-09 23:12:02.500195,0,https://g1.globo.com/,False,0c38b074a40bf19e69e1529def5b42636ce914be929bf3...,None
1,2025-09-09 23:12:05.338054,1,https://g1.globo.com/,True,db596588332cb0e98d18fdb8770cc6b806dc398f5bc6b0...,0c38b074a40bf19e69e1529def5b42636ce914be929bf3...
2,2025-09-09 23:12:08.166379,2,https://g1.globo.com/,True,06806f6b6b798b59dac9d9c201c7a15b98c04c1ed17cb9...,db596588332cb0e98d18fdb8770cc6b806dc398f5bc6b0...
3,2025-09-09 23:12:11.017222,3,https://g1.globo.com/,True,db596588332cb0e98d18fdb8770cc6b806dc398f5bc6b0...,06806f6b6b798b59dac9d9c201c7a15b98c04c1ed17cb9...
4,2025-09-09 23:12:13.848549,4,https://g1.globo.com/,False,db596588332cb0e98d18fdb8770cc6b806dc398f5bc6b0...,db596588332cb0e98d18fdb8770cc6b806dc398f5bc6b0...


,timestamp,attempt,url,changed,latest,previous
0,2025-09-09 23:12:16.038697,0,https://noticias.uol.com.br/,False,6e2446858e57c491c1881553913c147cbae05b493bf32d...,None
1,2025-09-09 23:12:18.093948,1,https://noticias.uol.com.br/,True,4c0380463fc4f8625fae567301070bf9493a26b41df499...,6e2446858e57c491c1881553913c147cbae05b493bf32d...
2,2025-09-09 23:12:20.144141,2,https://noticias.uol.com.br/,True,7a681ed4fc9789db70acd971f1213a65e4e056d9e069a1...,4c0380463fc4f8625fae567301070bf9493a26b41df499...
3,2025-09-09 23:12:22.189365,3,https://noticias.uol.com.br/,True,e6529d7c6c0532eaef88d900d0ce99303ebe1aba86bff5...,7a681ed4fc9789db70acd971f1213a65e4e056d9e069a1...
4,2025-09-09 23:12:24.228555,4,https://noticias.uol.com.br/,True,890be5d82cdb3c9d4183834c9b4366026bf6ef30e0cb7a...,e6529d7c6c0532eaef88d900d0ce99303ebe1aba86bff5...


,timestamp,attempt,url,changed,latest,previous
0,2025-09-09 23:12:26.524379,0,https://www.r7.com/,False,2370c944e035bcc3d352d7de743b8d66693b50253f4606...,None
1,2025-09-09 23:12:28.769885,1,https://www.r7.com/,True,b62ba540fb88cad6b1db0eba2d9bc1b380eb51c6bb4b63...,2370c944e035bcc3d352d7de743b8d66693b50253f4606...
2,2025-09-09 23:12:31.219531,2,https://www.r7.com/,True,9fb1dc70451f90d0849a1767fb032a84267eae34146b71...,b62ba540fb88cad6b1db0eba2d9bc1b380eb51c6bb4b63...
3,2025-09-09 23:12:33.366413,3,https://www.r7.com/,True,396de9db562c4da486a0605364fc66da4ace890eef4fa1...,9fb1dc70451f90d0849a1767fb032a84267eae34146b71...
4,2025-09-09 23:12:35.583251,4,https://www.r7.com/,True,1d25c8554d79188c7add2a7bdc127bca05e736828b22e7...,396de9db562c4da486a0605364fc66da4ace890eef4fa1...


,timestamp,attempt,url,changed,latest,previous
0,2025-09-09 23:12:37.915758,0,https://www.cnnbrasil.com.br/,False,4f0775ea7f765cb4040d4741d2c9f4c6c12e70dec5066f...,None
1,2025-09-09 23:12:40.667086,1,https://www.cnnbrasil.com.br/,True,d10dbdcffe84505df38a4f7d2f6de2eaa1b6fba1930fc6...,4f0775ea7f765cb4040d4741d2c9f4c6c12e70dec5066f...
2,2025-09-09 23:12:42.707278,2,https://www.cnnbrasil.com.br/,False,d10dbdcffe84505df38a4f7d2f6de2eaa1b6fba1930fc6...,d10dbdcffe84505df38a4f7d2f6de2eaa1b6fba1930fc6...
3,2025-09-09 23:12:44.891372,3,https://www.cnnbrasil.com.br/,True,f3f821bacb4f670b1bc461805cd848f6bae13b84be094c...,d10dbdcffe84505df38a4f7d2f6de2eaa1b6fba1930fc6...
4,2025-09-09 23:12:46.932106,4,https://www.cnnbrasil.com.br/,False,f3f821bacb4f670b1bc461805cd848f6bae13b84be094c...,f3f821bacb4f670b1bc461805cd848f6bae13b84be094c...


## 2 Parallel

In [5]:
from threading import Thread

In [6]:
# Task
def task(site: str, sites_data: list):
    site_monitor = sm.SiteMonitor()
    data = site_monitor.monitor_and_get_data(site)

    sites_data.append(data)

In [7]:
sites_data = []

start_time = perf_counter()

# 1 threads for each site
threads = []
for site in sm.SITES:
    threads.append(
        Thread(
            target = task,
            args = (site, sites_data)
        )
    )

# Inicializa as 4 threads
for thread in threads:
    thread.start()

# Aguarda até que as 4 threads sejam completadas
for thread in threads:
    thread.join()

end_time = perf_counter()

parallel_elapsed_time = end_time - start_time
print(f'\033[As tarefas levaram {parallel_elapsed_time: 0.2f} segundo(s) para executar.\033[0m')

/workspaces/parallel-and-distribute-computing/unit2/exercises/1-site-monitoring/../site_monitoring.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._data = pd.concat([self._data, pd.DataFrame({


[2025-09-09 23:12:49.169831] Monitoring attempt: (0) Url: https://noticias.uol.com.br/ Did change?: False | Hashes latest: a76346fc229f7807fdf52cde96fd890ed4cf7ac55714273bd625ed6e VS. previous: None
[2025-09-09 23:12:49.206195] Monitoring attempt: (0) Url: https://www.r7.com/ Did change?: False | Hashes latest: d116b4ba80d1884ce6e11ebc0604a39362efc63e09b3b9f13f389b9d VS. previous: None
[2025-09-09 23:12:49.238099] Monitoring attempt: (0) Url: https://www.cnnbrasil.com.br/ Did change?: False | Hashes latest: 3fde0fe48b3bdbe309382e3c37ae304bbf9b339ff28ebbc94d3a4ac8 VS. previous: None
[2025-09-09 23:12:49.821363] Monitoring attempt: (0) Url: https://g1.globo.com/ Did change?: False | Hashes latest: 5ea81c7c3c3ade733f040f74f2cc310dcec0bd48ef8629e3e7437879 VS. previous: None
[2025-09-09 23:12:51.214023] Monitoring attempt: (1) Url: https://noticias.uol.com.br/ Did change?: True | Hashes latest: 6b1415c0678caa07451c3e007f784197bbd242df2350b0f87d53c009 VS. previous: a76346fc229f7807fdf52cde96

In [8]:
for data in sites_data:
    display(data)

,timestamp,attempt,url,changed,latest,previous
0,2025-09-09 23:12:49.167974,0,https://noticias.uol.com.br/,False,a76346fc229f7807fdf52cde96fd890ed4cf7ac5571427...,None
1,2025-09-09 23:12:51.212640,1,https://noticias.uol.com.br/,True,6b1415c0678caa07451c3e007f784197bbd242df2350b0...,a76346fc229f7807fdf52cde96fd890ed4cf7ac5571427...
2,2025-09-09 23:12:53.252450,2,https://noticias.uol.com.br/,True,2a278d71b2809eceacba0902d548f33a325fa558fcda98...,6b1415c0678caa07451c3e007f784197bbd242df2350b0...
3,2025-09-09 23:12:55.295095,3,https://noticias.uol.com.br/,True,334b77abe3a7cfe6a65eb4a6f37bb94912ae1c5c5a6fab...,2a278d71b2809eceacba0902d548f33a325fa558fcda98...
4,2025-09-09 23:12:57.336718,4,https://noticias.uol.com.br/,True,ec56a50528af997b7c1efee741918f964738a9d0efb71e...,334b77abe3a7cfe6a65eb4a6f37bb94912ae1c5c5a6fab...


,timestamp,attempt,url,changed,latest,previous
0,2025-09-09 23:12:49.237072,0,https://www.cnnbrasil.com.br/,False,3fde0fe48b3bdbe309382e3c37ae304bbf9b339ff28ebb...,None
1,2025-09-09 23:12:51.278079,1,https://www.cnnbrasil.com.br/,False,3fde0fe48b3bdbe309382e3c37ae304bbf9b339ff28ebb...,3fde0fe48b3bdbe309382e3c37ae304bbf9b339ff28ebb...
2,2025-09-09 23:12:53.320260,2,https://www.cnnbrasil.com.br/,False,3fde0fe48b3bdbe309382e3c37ae304bbf9b339ff28ebb...,3fde0fe48b3bdbe309382e3c37ae304bbf9b339ff28ebb...
3,2025-09-09 23:12:55.612076,3,https://www.cnnbrasil.com.br/,True,db021d5b66b4763f21e6ae36ea7fd39a36099b22793dd3...,3fde0fe48b3bdbe309382e3c37ae304bbf9b339ff28ebb...
4,2025-09-09 23:12:57.655249,4,https://www.cnnbrasil.com.br/,False,db021d5b66b4763f21e6ae36ea7fd39a36099b22793dd3...,db021d5b66b4763f21e6ae36ea7fd39a36099b22793dd3...


,timestamp,attempt,url,changed,latest,previous
0,2025-09-09 23:12:49.204497,0,https://www.r7.com/,False,d116b4ba80d1884ce6e11ebc0604a39362efc63e09b3b9...,None
1,2025-09-09 23:12:51.411855,1,https://www.r7.com/,True,be8cad7868d4d28f8b8476bac5dde3ff8e07053c54ed10...,d116b4ba80d1884ce6e11ebc0604a39362efc63e09b3b9...
2,2025-09-09 23:12:53.603348,2,https://www.r7.com/,True,db74d6135f9591a4fb0464d1da4fc386ead28ad0fa9b7c...,be8cad7868d4d28f8b8476bac5dde3ff8e07053c54ed10...
3,2025-09-09 23:12:55.782220,3,https://www.r7.com/,True,eed24c6d935675d126a5a88e56bb8e17c2e64e4549fbd3...,db74d6135f9591a4fb0464d1da4fc386ead28ad0fa9b7c...
4,2025-09-09 23:12:58.035193,4,https://www.r7.com/,True,8c3e59c10e165213a8045bd72f647d355408ddd0e5c17b...,eed24c6d935675d126a5a88e56bb8e17c2e64e4549fbd3...


,timestamp,attempt,url,changed,latest,previous
0,2025-09-09 23:12:49.820072,0,https://g1.globo.com/,False,5ea81c7c3c3ade733f040f74f2cc310dcec0bd48ef8629...,None
1,2025-09-09 23:12:52.646689,1,https://g1.globo.com/,True,69b2b8b6a71648d11d9ad02d356b91b2fda9fa51b2fb69...,5ea81c7c3c3ade733f040f74f2cc310dcec0bd48ef8629...
2,2025-09-09 23:12:55.472551,2,https://g1.globo.com/,True,5ea81c7c3c3ade733f040f74f2cc310dcec0bd48ef8629...,69b2b8b6a71648d11d9ad02d356b91b2fda9fa51b2fb69...
3,2025-09-09 23:12:58.299345,3,https://g1.globo.com/,False,5ea81c7c3c3ade733f040f74f2cc310dcec0bd48ef8629...,5ea81c7c3c3ade733f040f74f2cc310dcec0bd48ef8629...
4,2025-09-09 23:13:01.127207,4,https://g1.globo.com/,True,644121a06a0a3c6a9c9d3d6b16cdf53f535a4f6f159519...,5ea81c7c3c3ade733f040f74f2cc310dcec0bd48ef8629...


## 3 Compare

In [9]:
print(f'Speedup: {sequential_elapsed_time / parallel_elapsed_time: 0.2f}x')

Speedup:  3.34x
